In [1]:
import gmsh
import numpy as np
import FuncionesMEF3D as M3D

In [2]:
gmsh.initialize()
gmsh.model.add('Cubo Prueba')

In [3]:
L=1 #m
lc=L
nu=0.3
E=210e9 #Pa

p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(L, 0, 0, lc)
p3 = gmsh.model.geo.addPoint(L, L, 0, lc)
p4 = gmsh.model.geo.addPoint(0, L, 0, lc)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

C = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4])

S = gmsh.model.geo.addPlaneSurface([C])

V_cubo = gmsh.model.geo.extrude([(2,S)],0,0,L)

gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(dim=3)
gmsh.model.geo.synchronize()
# gmsh.fltk.run()

In [4]:
NodeInfo = gmsh.model.mesh.get_nodes() #saco un objeto con todos los nodos. array uno etiquetas que asigna a los nodos, empieza con 1 y no con 0.
NN = NodeInfo[0].shape[0] #numero de nodos es la cantidad de elementos uqe tengo en el primer array
NN_Tags = NodeInfo[0]-1 #Los Tags de los nodos los pongo en numeración python.
MN = NodeInfo[1].reshape(NN,3)

E_Tags, MC_Flatten = gmsh.model.mesh.get_elements_by_type(4) #dame los tags (numero de elementos) del tipo 4 (tetrahedros) la MC flatten.
NNXE=4
NE=E_Tags.shape[0]
MC=MC_Flatten.reshape(NE,4)
MC=MC-np.ones(MC.shape)#Lo dejo en numeración python

In [5]:
NE

24

Para calcular los desplazamientos u(x,y,z) debo conocer V, alpha_i,beta_i, gamma_i y delta_i (i=1,2,3,4) según la ecuacion:

11.2.3

Análogamente, se obtienen v(x,y,z) y w(x,y,z)

Para obtener 6V se evalua el determinante:

11.2.4

donde xi,yi,zi , i=1,2,3,4
son las coordenadas del nodo i. Como el Volumen de todos los tetraedros es iguales, me fijo para el primer elemento los nodos que lo componen (MC) y para esos nodos busco sus posiciones (MN).


In [6]:
def k_elemental_3D(MN, MC, nu, E, elemento):
    
    Fila_MC = MC[elemento, :].astype(int)
    x1, y1, z1= MN[Fila_MC[0]][0], MN[Fila_MC[0]][1], MN[Fila_MC[0]][2]
    x2, y2, z2= MN[Fila_MC[1]][0], MN[Fila_MC[1]][1], MN[Fila_MC[1]][2]
    x3, y3, z3= MN[Fila_MC[2]][0], MN[Fila_MC[2]][1], MN[Fila_MC[2]][2]
    x4, y4, z4= MN[Fila_MC[3]][0], MN[Fila_MC[3]][1], MN[Fila_MC[3]][2]

    SeisV=np.array([[1,x1,y1,z1],
                    [1,x2,y2,z2],
                    [1,x3,y3,z3],
                    [1,x4,y4,z4]])
    
    V=np.abs(np.linalg.det(SeisV)/6)

    D=np.array([[1-nu,nu,nu,0,0,0],
       [nu,1-nu,nu,0,0,0],
       [nu,nu,1-nu,0,0,0],
       [0,0,0,(1-2*nu)/2,0,0],
       [0,0,0,0,(1-2*nu)/2,0],
       [0,0,0,0,0,(1-2*nu)/2]])*(E/((1+nu)*(1-2*nu)))
    
    Coeficientes=np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            Coeficientes[i, j]=M3D.Menores_Det(SeisV, i, j)

    alpha1 = Coeficientes[0,0]
    alpha2 = -Coeficientes[1,0]
    alpha3 = Coeficientes[2,0]
    alpha4 = -Coeficientes[3,0]

    beta1 = -Coeficientes[0,1]
    beta2 = Coeficientes[1,1]
    beta3 = -Coeficientes[2,1]
    beta4 = Coeficientes[3,1]

    gamma1 = Coeficientes[0,2]
    gamma2 = -Coeficientes[1,2]
    gamma3 = Coeficientes[2,2]
    gamma4 = -Coeficientes[3,2]

    delta1 = -Coeficientes[0,3]
    delta2 = Coeficientes[1,3]
    delta3 = -Coeficientes[2,3]
    delta4 = Coeficientes[3,3]

    B1=np.array([[beta1,0,0],
                 [0,gamma1,0],
                 [0,0,delta1],
                 [gamma1,beta1,0],
                 [0,delta1,gamma1],
                 [delta1,0,beta1]])

    B2=np.array([[beta2,0,0],
                 [0,gamma2,0],
                 [0,0,delta2],
                 [gamma2,beta2,0],
                 [0,delta2,gamma2],
                 [delta2,0,beta2]])

    B3=np.array([[beta3,0,0],
                 [0,gamma3,0],
                 [0,0,delta3],
                 [gamma3,beta3,0],
                 [0,delta3,gamma3],
                 [delta3,0,beta3]])

    B4=np.array([[beta4,0,0],
                 [0,gamma4,0],
                 [0,0,delta4],
                 [gamma4,beta4,0],
                 [0,delta4,gamma4],
                 [delta4,0,beta4]])

    B=np.hstack([B1,B2,B3,B4])/(6*V)
    Kel=np.transpose(B).dot(D.dot(B))*V
    # print(Kel)
    return Kel, D, B

In [7]:
GLXN = 3
for e in range(NE):
    Kel, D, B = k_elemental_3D(MN, MC, nu, E, e)
    K = M3D.Ensamblado_Matriz_Global(MN, MC, Kel, GLXN)

In [8]:
Traccionado_PG = gmsh.model.addPhysicalGroup(2, [26])
gmsh.model.geo.synchronize()

In [9]:
# gmsh.fltk.run()

Pruebo solicitando el cubo a traccion

In [10]:
EntidadTraccionado=gmsh.model.getEntitiesForPhysicalGroup(2,Traccionado_PG)
ETypesTraccionado, ETagsTraccionado, NodeTagsTraccionado = gmsh.model.mesh.getElements(2,EntidadTraccionado[0]) 
NElementosTraccionados=len(ETagsTraccionado[0])
MC_T=NodeTagsTraccionado[0].reshape(NElementosTraccionados,GLXN)-1


s=(np.array([1,2,3,4,9])-1)*GLXN+2
Us=np.zeros_like(s)
r = np.array([i for i in range(NN*GLXN) if i not in s])
Fr=np.zeros_like(r).astype(float)
Tension=100

for eT in range(NElementosTraccionados):
    n1=MC_T[eT,0].astype(int)
    n2=MC_T[eT,1].astype(int)
    n3=MC_T[eT,2].astype(int)

    A= (1/2)*np.abs(np.linalg.det(np.array([ [MN[n1,0] ,MN[n1,1], 1], 
                                             [MN[n2,0] ,MN[n2,1], 1], 
                                             [MN[n3,0] ,MN[n3,1], 1]])))
    print('nodos: ', n1*GLXN+2, n2*GLXN+2, n3*GLXN+2)
    Fr[r==n1*GLXN+2] += Tension * A / 3
    Fr[r==n2*GLXN+2] += Tension * A / 3
    Fr[r==n3*GLXN+2] += Tension * A / 3

nodos:  14 17 41
nodos:  23 14 41
nodos:  17 20 41
nodos:  20 23 41


In [11]:
# 4 5 6 7 13
print(s)
print(r)
print(Fr.flatten())

[ 2  5  8 11 26]
[ 0  1  3  4  6  7  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 25 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         16.66666667  0.
  0.         16.66666667  0.          0.         16.66666667  0.
  0.         16.66666667  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 33.33333333]


In [12]:
U,F=M3D.solve(K,s,r,Us,Fr)

In [13]:
U3D = U.reshape(NN,GLXN)

In [14]:
strains = gmsh.view.add("Desplazamientos")
# por algun motivo le faltaba sumar 1 a nodeinfo
strain_model_data = gmsh.view.addModelData(strains, 0, 'Cubo Prueba', 'NodeData', NodeInfo[0], U3D, numComponents=3)
gmsh.option.setNumber(f'View[{strains}].VectorType',5)

F3D = F.reshape(NN,GLXN)

forces = gmsh.view.add('forces')
# por algun motivo le faltaba sumar 1 a nodeinfo
forces_model_data = gmsh.view.addModelData(forces, 0, 'Cubo Prueba','NodeData',NodeInfo[0], F3D, numComponents=3)
gmsh.option.setNumber(f'View[{forces}].VectorType',4)
gmsh.option.setNumber(f'View[{forces}].GlyphLocation',2)
# gmsh.fltk.run()

gmsh.fltk.run()